In [ ]:
###General Code First Box
import pandas as pd
import numpy as np
import scanpy as sc
import magpy as mp
import scipy.stats as stats

def sort_list_by_pctexp(adata, gene_list, groupby='lineage'):
    subset = adata[:,gene_list]
    clusters = subset.obs[groupby].unique()
    pctexp = np.zeros((len(clusters),subset.shape[1]))
    for i,cluster in enumerate(clusters):
        subset2 = subset[subset.obs[groupby]==cluster]
        pctexp[i] = (subset2.layers['raw_normalized']>0).sum(axis=0).A1 / subset.shape[0]
    subset.var['pctmax'] = pctexp.max(axis=0)
    sorted_list = subset.var['pctmax'].sort_values(ascending=False).index.tolist()
    return sorted_list
def sort_list_by_mean(adata, gene_list, groupby='lineage'):
    subset = adata[:,gene_list]
    clusters = subset.obs[groupby].unique()
    cmeans = np.zeros((len(clusters),subset.shape[1]))
    for i,cluster in enumerate(clusters):
        subset2 = subset[subset.obs[groupby]==cluster]
        cmeans[i] = subset2.layers['raw_normalized'].mean(axis=0).A1
    subset.var['cmeans'] = cmeans.max(axis=0)
    sorted_list = subset.var['cmeans'].sort_values(ascending=False).index.tolist()
    return sorted_list
def means_to_df(adata, gene_list, save=None, groupby='lineage'):
    subset = adata[:,gene_list]
    clusters = subset.obs[groupby].unique()
    cmeans = {}
    for i,cluster in enumerate(clusters):
        subset2 = subset[subset.obs[groupby]==cluster]
        cmeans[cluster] = subset2.layers['raw_normalized'].mean(axis=0).A1
    df = pd.DataFrame(cmeans,index=gene_list)
    if save: df.to_csv(save)
    return df

combined_path = "/proj/magness/CDS010-014-015_combined"

sc.set_figure_params(dpi_save = 300, format = 'svg')
fig_path = combined_path+'fig5-6-MarkerDEGs/'
sc.settings.figdir = fig_path

In [ ]:
###Fig 6
###UMAPS for DEGs from each lineage
adata = mp.load(combined_path, "clustered_annotated_adata_k25_lr0.92_v1.5.h5ad")

gene_list = mp.filter_genes(adata, ['LGR5','DEFA5','DEFA6','PRSS2','REG3A','MT1G','CPA2','SULT1E1','ALDOB','MTTP','APOA4','RBP2','SLC5A1','ABCA4','REG4','SLC16A7','MELTF','GABRP','CIART','CHGA','CHGB','CCK','GNPTAB','CPNE8','SLC26A2','FABP5','CA12','SLC38A1','MUC5B','MUC1','MUC4','WFDC2','TNFRSF11B','BEST2','AFAP1L2','TIMP1','ITGB5','PYY','GCG'
])

sc.set_figure_params(dpi_save = 300, format = 'svg')
fig_path = '/home/joseph/'
sc.settings.figdir = fig_path+'ISC-TA-Panels/'
sc.pl.umap(adata,color=gene_list, ncols=5,cmap=mp.reds, save='ISC_TA_competence_umaps')